# Wahlkreiskarten

### Die hier erzeugten Karten dienen später im Wahlkreisfinder zur Orientierung und helfen bei der Auswahl eines Wahlkreises

Dieses Notebook erzeugt Bilder der Wahlkreise, respektive mehrere Wahlkreise zusammen.

Das entsprechende GeoJson entstammt den Repository



In [1]:
import json
from descartes import PolygonPatch
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import gmplot
from selenium import webdriver 

In [2]:
# Colors of the different parties. Used to mark the boundaries in the leading color
color_dict = {
    "Sonstige": "grey",
    "DIE LINKE" :  "#96276E",
    "FDP":"#F6BB00",
    "AfD":"#34A3D2",
    "SPD":"#DB4240",
    "CSU":"#373737",
    "CDU":"#373737",
    "CDU/CSU": '#373737',
    "GRÜNE":"#4BA345",
    "PIRATE": "#FF8800"
}

In [3]:
# Open files used to create polygones
with open("wahlkreise.geojson.txt") as json_file:
    datawk = json.load(json_file) # or geojson.load(json_file)
    
with open('wahlkreis_info.json') as f:
    wkinfo = json.load(f)

This part creates an html file based on googlemaps. It plots multigon shapes

In [4]:
for nr in range(0,299):
    
    #get some info ( it think, i did not use these)
    list_plz = wkinfo['districts'][nr]['plz']
    wk_name = wkinfo['districts'][nr]['district']
    wk_nr = wkinfo['districts'][nr]['district_id']
    party_color = wkinfo['districts'][nr]['election_13']
    party_color['wahlbeteiligung'] = 0
    color_bound = color_dict[max(party_color, key=party_color.get)]
    
        
    ploywk = datawk['features'][nr]['geometry']
    xma = 0
    xmi = 100
    yma = 0
    ymi = 100
    for item1 in ploywk['coordinates']:
        for item3 in item1:
            try:
                #for item3 in item2 polygon:
                if item3[0] > xma:
                    xma = item3[0]
                if item3[0] < xmi:
                    xmi = item3[0]
                if item3[1] > yma:
                    yma = item3[1]
                if item3[1] < ymi:
                    ymi = item3[1]
            except:
                # multigon
                item2 = item3
                for item3 in item2:
                    if item3[0] > xma:
                        xma = item3[0]
                    if item3[0] < xmi:
                        xmi = item3[0]
                    if item3[1] > yma:
                        yma = item3[1]
                    if item3[1] < ymi:
                        ymi = item3[1]
    #set up the box for taking into account few percent space to the boarders
    adjustx = 0.05*(xma-xmi)
    adjusty = 0.05*(yma-ymi)
    centerx = (xma+xmi)/2
    centery= (yma+ymi)/2
    bound_box = [xmi-adjustx, xma+adjustx, ymi-adjusty, yma+adjusty]

    # fix zoom-level in google maps
    xlen = xma-xmi
    ylen = yma-ymi
    if xlen < 0.45 and ylen < 0.175:
        zoom = 12
    elif xlen< 0.9 and ylen < 0.35:
        zoom = 11
    elif xlen < 1.9 and ylen < 0.7:
        zoom = 10
    elif xlen < 3.7 and ylen < 1.4:
        zoom = 9
    else:
        zoom = 8
    
    # create google map
    gmap = gmplot.GoogleMapPlotter(centery, centerx, zoom)

    
    polywk = datawk['features'][nr]['geometry']
    coords_all = polywk['coordinates']
    try:
        #multigon
        for coords in coords_all:
            x = [i for i,j in coords[0]]
            y = [j for i,j in coords[0]]
            #xm,ym = m(x,y)
            gmap.plot(y, x, color = color_bound, edge_width=10)
    except:
        #polygon
        coords = coords_all
        x = [i for i,j in coords[0]]
        y = [j for i,j in coords[0]]
        gmap.plot(y, x, color = color_bound, edge_width=10)
        
    #Safe as html
    gmap.draw("./fig/wk"+str(nr+1)+".html")

This part opens the html file and takes a screenshot

In [5]:
for nr in range(0,299):

    # get the name into the right format
    temp_name = '.fig/wk'+str(nr)+'.html'

    # open in webpage
    driver = webdriver.PhantomJS()
    driver.set_window_size(1280,800) # set the window size that you need 
    driver.get(temp_name)
    # BUILD SOMETHING, SUCH THAT IT WAITS TILL PAGE IS FULLY LOADED
    save_name = 'wk'+str(nr)+'.png'       
    driver.save_screenshot('./fig/'+save_name)
    driver.quit()


NameError: name 'Path' is not defined

Done. Thx.